<a href="https://colab.research.google.com/github/MonyaB117/SUMMER_ANALYTICS-PROJECT/blob/main/FINAL_HACK_2_SUBM_MONYA_BALANI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

IMPORTING ALL THE IMPORTANT LIBRARIES

In [ ]:
pip install catboost

In [ ]:
import numpy as np
import pandas as pd
import os
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from catboost import CatBoostClassifier
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split ,cross_val_score, StratifiedKFold
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt

LOADING THE TRAINING AND TESTING DATA

In [ ]:
df_train = pd.read_csv("https://raw.githubusercontent.com/MonyaB117/SUMMER_ANALYTICS-PROJECT/refs/heads/main/Train_Data.csv")
df_test = pd.read_csv('https://raw.githubusercontent.com/MonyaB117/SUMMER_ANALYTICS-PROJECT/refs/heads/main/Test_Data.csv')

PREPROCESSING THE TRAINING DATA

In [ ]:
# Fill missing values with column mean
for col in ['BMXBMI', 'LBXGLU', 'LBXGLT', 'RIAGENDR', 'LBXIN']:
    df_train[col].fillna(df_train[col].mean(), inplace=True)

# Drop rows with essential NaNs
df_train.dropna(subset=['DIQ010', 'age_group', 'PAQ605', 'SEQN'], inplace=True)
df_train.reset_index(drop=True, inplace=True)

# Encode age_group
df_train['age_group'] = df_train['age_group'].replace({'Adult': 0, 'Senior': 1})

/tmp/ipython-input-69-2950222361.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_train[col].fillna(df_train[col].mean(), inplace=True)
/tmp/ipython-input-69-2950222361.py:10: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_train['age_group'] = df_train['age_group'].replace({'Adult': 0, 'Seni

PREPROCESSING THE TESTING DATA

In [ ]:
# Fill missing values with column mean
for col in ['BMXBMI', 'LBXGLU', 'LBXGLT', 'RIAGENDR', 'LBXIN', 'DIQ010', 'PAQ605', 'SEQN']:
    df_test[col].fillna(df_test[col].mean(), inplace=True)



/tmp/ipython-input-70-1952601329.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_test[col].fillna(df_test[col].mean(), inplace=True)


FEATURE SELECTION AND CHECKING WHICH MODEL IS EFFICIENT FOR TRAINING THE DATSET

In [ ]:
features = ['BMXBMI', 'LBXGLU', 'LBXGLT', 'RIAGENDR', 'LBXIN']
X_train_final = df_train[features]
y_train_final = df_train['age_group']
X_test_final = df_test[features]

DEFINING 7 FOLD STRATIFIED CROSS VALIDATION

In [ ]:
skf = StratifiedKFold(n_splits=7, shuffle=True, random_state=42)

APPLYING SVC MODEL WITH LINEAR KERNEL

In [ ]:
svc_model = SVC(kernel='linear')
svc_scores = cross_val_score(svc_model, X_train_final, y_train_final, cv=skf, scoring='accuracy')
print(f"SVC Mean CV Accuracy: {np.mean(svc_scores):.4f} | Std: {np.std(svc_scores):.4f}")
print(f"SVC CROSS VALIDATION SCORE: ", svc_scores)

SVC Mean CV Accuracy: 0.8381 | Std: 0.0012
SVC CROSS VALIDATION SCORE:  [0.83882784 0.83882784 0.83882784 0.83882784 0.83516484 0.83823529
 0.83823529]


APPLYING CATBOOST MODEL (SINCE MOST OF THE TRAINING DATA IS IN CATEGORIES)

In [ ]:
# CatBoost
cat_model = CatBoostClassifier(verbose=0, random_state=42)
cat_scores = cross_val_score(cat_model, X_train_final, y_train_final, cv=skf, scoring='accuracy')
print(f"CatBoost Mean CV Accuracy: {np.mean(cat_scores):.4f} | Std: {np.std(cat_scores):.4f}")
print(f"CATBOOST CROSS VALIDATION SCORE: ", cat_scores)

CatBoost Mean CV Accuracy: 0.8407 | Std: 0.0165
CATBOOST CROSS VALIDATION SCORE:  [0.81684982 0.84615385 0.83516484 0.86080586 0.86080586 0.84558824
 0.81985294]


APPLYING ADABOOST MODEL

In [ ]:
# AdaBoost
ada_model = AdaBoostClassifier(random_state=42)
ada_scores = cross_val_score(ada_model, X_train_final, y_train_final, cv=skf, scoring='accuracy')
print(f"AdaBoost Mean CV Accuracy: {np.mean(ada_scores):.4f} | Std: {np.std(ada_scores):.4f}")
print(f"ADABOOST CROSS VALIDATION SCORE: ", ada_scores)

AdaBoost Mean CV Accuracy: 0.8334 | Std: 0.0137
ADABOOST CROSS VALIDATION SCORE:  [0.83516484 0.83150183 0.83882784 0.83882784 0.84249084 0.84558824
 0.80147059]


APPLYING GRADIENTBOOST MODEL

In [ ]:
# Gradient Boosting
gb_model = GradientBoostingClassifier(random_state=42)
gb_scores = cross_val_score(gb_model, X_train_final, y_train_final, cv=skf, scoring='accuracy')
print(f"Gradient Boosting Mean CV Accuracy: {np.mean(gb_scores):.4f} | Std: {np.std(gb_scores):.4f}")
print(f"GRADIENTBOOST CROSS VALIDATION SCORE: ", gb_scores)

Gradient Boosting Mean CV Accuracy: 0.8334 | Std: 0.0208
GRADIENTBOOST CROSS VALIDATION SCORE:  [0.82051282 0.82417582 0.82417582 0.85347985 0.86813187 0.84191176
 0.80147059]


TRAIN , TEST AND SPLIT FOR CHECKING THE FINAL ACCURACY

In [ ]:
# Train-Test Split for final accuracy on holdout set
X_train, X_test, y_train, y_test = train_test_split(X_train_final, y_train_final, test_size=0.2, stratify=y_train_final, random_state=42)

ACCURACY FOR SVC MODEL

In [ ]:
# SVC Accuracy
svc_model.fit(X_train, y_train)
y_pred_svc = svc_model.predict(X_test)
svc_acc = accuracy_score(y_test, y_pred_svc)
print(f"SVC Holdout Accuracy: {svc_acc:.4f}")


SVC Holdout Accuracy: 0.8377


ACCURACY FOR CATBOOST MODEL

In [ ]:
# CatBoost Accuracy
cat_model.fit(X_train, y_train)
y_pred_cat = cat_model.predict(X_test)
cat_acc = accuracy_score(y_test, y_pred_cat)
print(f"CatBoost Holdout Accuracy: {cat_acc:.4f}")

CatBoost Holdout Accuracy: 0.8037


ACCRACY FOR ADABOOST MODEL

In [ ]:
# AdaBoost Accuracy
ada_model.fit(X_train, y_train)
y_pred_ada = ada_model.predict(X_test)
ada_acc = accuracy_score(y_test, y_pred_ada)
print(f"AdaBoost Holdout Accuracy: {ada_acc:.4f}")


AdaBoost Holdout Accuracy: 0.8298


ACCURACY FOR GRADIENT BOOST MODEL

In [ ]:
# Gradient Boosting Accuracy
gb_model.fit(X_train, y_train)
y_pred_gb = gb_model.predict(X_test)
gb_acc = accuracy_score(y_test, y_pred_gb)
print(f"Gradient Boosting Holdout Accuracy: {gb_acc:.4f}")

Gradient Boosting Holdout Accuracy: 0.8194


AFTER LOOKING ON THE CROSS VALIDATION SCORES, ACCURACY SCORES AND THE TYPE OF DATA WE HAVE IN THE TEST DATASET , WE WILL BE APPLYING CATBOOST MODEL AND STORE THE OUTPUT IN THE FINAL_SUBMISSION_FILE

In [ ]:
final_catboost = CatBoostClassifier(verbose=0, random_state=42)
final_catboost.fit(X_train_final, y_train_final)
pred_submission = final_catboost.predict(X_test_final)

HACK_2_FINAL_submission = pd.DataFrame({
    'age_group':pred_submission

})

print("\nSubmission Preview:")
print(HACK_2_FINAL_submission)

HACK_2_FINAL_submission.to_csv('Final_Predicted_Submission.csv', index=False)


print("Submission file saved as Final_Predicted_Submission.csv")


Submission Preview:
     age_group
0            0
1            0
2            0
3            0
4            0
..         ...
307          0
308          0
309          0
310          0
311          0

[312 rows x 1 columns]
Submission file saved as Final_Predicted_Submission.csv
